In [ ]:
!pip install soundfile

In [ ]:
import os
import numpy as np
import pandas as pd
import soundfile as sf
import scipy.signal as signal
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from keras.optimizers import Adam
#from keras.models import Sequential
#from keras.layers import Dense, Activation, Bidirectional,Attention

In [ ]:
train_path = '../input/train/train/'
test_path = '../input/test/test/'

In [ ]:
filename = 'de_f_0809fd0642232f8c85b0b3d545dc2b5a.fragment1.flac'

In [ ]:
data, samplerate = sf.read(train_path+filename)

In [ ]:
data

In [ ]:
Pxx, freqs, bins, im = plt.specgram(data, Fs=samplerate)

# add axis labels
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')

In [ ]:
data

In [ ]:
Time= np.linspace(0, len( data )/samplerate, num=len(data))
plt.plot(Time, data )
plt.ylabel('Amplitude ')
plt.xlabel('Time ')

In [ ]:
label = []
for filename in os.listdir(train_path)[:100]:
    label.append(filename[:2])

In [ ]:
gender = []
for filename in os.listdir(train_path)[:100]:
    gender.append('male' if filename[3:4]=='m' else 'female')

In [ ]:
sound_type = []
for filename in os.listdir(train_path)[:100]:
    sound_type.append('noise' if 'noise' in filename else 'pitch' if 'pitch' in filename else 'speed' if 'speed' in filename else 'notype')

In [ ]:
file = []
for filename in os.listdir(train_path)[:100]:
    file.append(filename)

In [ ]:
series = []
length = []
for filename in os.listdir(train_path)[:100]:
    flac, samplerate = sf.read(train_path+filename)
    series.append(flac)
    length.append(samplerate)

In [ ]:
data = {'Gender':gender,
        'filename':file,
           'Sound_type': sound_type,
       'language':label,
       'series': series,
       'length': length }

In [ ]:
df = pd.DataFrame( data )

In [ ]:
df

In [ ]:
len(df)

In [ ]:
df_male_en = df[(df['Gender']=='male') & (df['language']=='en')].reset_index(drop=True)
df_male_es = df[(df['Gender']=='male') & (df['language']=='es')].reset_index(drop=True)
df_male_de = df[(df['Gender']=='male') & (df['language']=='de')].reset_index(drop=True)
df_female_en = df[(df['Gender']=='female') & (df['language']=='en')].reset_index(drop=True)
df_female_es = df[(df['Gender']=='female') & (df['language']=='es')].reset_index(drop=True)
df_female_de = df[(df['Gender']=='female') & (df['language']=='de')].reset_index(drop=True)

In [ ]:
fig, ax = plt.subplots(2, 4, figsize = (12, 16))
for i in range(8):
    ax[i//4, i%4].plot(df_male_en['series'][i])
    ax[i//4, i%4].set_title(df_male_en['filename'][i][:-4], fontsize=5)
    ax[i//4, i%4].get_xaxis().set_ticks([])

The first 8 male with espanol language waveform

In [ ]:
df.columns

Splitting data for training and testing using SciKit-learn

In [ ]:
df.dtypes

In [ ]:
df.language.replace({'en':1,'es':2,'de':3},inplace=True)

df.language.unique()

In [ ]:
df

In [ ]:
y=df.language

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split


files = np.array(file)
y = np.array(y)
paths_train, paths_valid, y_train, y_valid = train_test_split(files, y, test_size = 0.2, random_state=45)
print(paths_train.shape, y_train.shape, paths_valid.shape, y_valid.shape)


In [ ]:
from keras.layers import Embedding,ELU, Flatten, Input, Dense, CuDNNGRU,concatenate, Bidirectional, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None
    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
# Neural network model
input_shape = (636,128)
optimizer = Adam(0.005, beta_1=0.1, beta_2=0.001, amsgrad=True)
n_classes = 80

model = Sequential()
model.add(Bidirectional(CuDNNLSTM(256, return_sequences=True), input_shape=input_shape))
model.add(Attention(636))
model.add(Dropout(0.2))

model.add(Dense(400))
model.add(ELU())
model.add(Dropout(0.2)) 

model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])
